In [25]:
!python ../src/utils.py

In [26]:
%reload_ext autoreload

In [27]:
import os
import ROOT as rt
import numpy as np
import matplotlib.pyplot as plt
import utils 
import array
from ROOT import VecOps
from analysis import (df, df_filtered, COLOR_MAP)


name_tag = str(utils.TAG)
print("cwd:", os.getcwd())

cwd: /home/student/projects/optimizing_DEDx_estimator/notebooks


In [28]:
f = rt.TFile(f"{utils.OUTPUT_ROOT}/2025-06-23_histos.root") #opens up the root file for reading

!rootls -t ../output/root/2025-06-23_histos.root

TCanvas  Jun 23 11:01 2025 h2_canvas_overlay;1             "Log-scale overlay" 
TCanvas  Jun 23 11:02 2025 h2_filtered_canvas_overlay;1    "Log-scale overlay" 
THStack  Jun 23 11:02 2025 h2_filtered_overlay;1           "Generalized harmonic mean (HLT_Mu50 with Isotrack_pt>55)" 
THStack  Jun 23 11:01 2025 h2_overlay;1                    "Generalized harmonic means" 
THStack  Jun 23 11:08 2025 Trunc_filtered_overlay;1        "Truncated mean (HLT_Mu50 with Isotrack_pt>55)" 
TCanvas  Jun 23 11:08 2025 Trunc_filtered_overlay_canvas;1 "Log-scale overlay" 
THStack  Jun 23 11:07 2025 Trunc_overlay;1                 "Truncated mean" 
TCanvas  Jun 23 11:07 2025 Trunc_overlay_canvas;1          "Log-scale overlay" 


In [29]:
h2_stack = f.Get("h2_overlay")
c = f.Get("h2_canvas_overlay")

h2_stack.Draw("nostack")
c.Draw()

In [30]:
h2_stack = f.Get("h2_overlay")
h2_canvas = f.Get("h2_canvas_overlay")
h2_hist_list = h2_stack.GetHists()


h2_filtered_stack = f.Get("h2_filtered_overlay")
h2_filtered_canvas = f.Get("h2_filtered_canvas_overlay")
h2_filtered_hist_list = h2_filtered_stack.GetHists()


trunc_stack = f.Get("Trunc_overlay")
trunc_canvas = f.Get("Trunc_overlay_canvas")
trunc_hist_list = trunc_stack.GetHists()


trunc_filtered_stack = f.Get("Trunc_filtered_overlay")
trunc_filtered_canvas = f.Get("Trunc_filtered_overlay_canvas")
trunc_filtered_hist_list = trunc_filtered_stack.GetHists()

In [31]:
h_num = h2_hist_list.At(0)
h_den = h2_hist_list.At(3)

In [32]:
%jsroot on
drawer = utils.HistogramDrawer()
rp = drawer.make_two_ratio(h_num, h_den)
c = drawer.draw_ratio(rp, canvas_name="ratio", title="My Ratio", logy = True)
c.Draw()


Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 43 points have been skipped


In [ ]:
def create_two_ratio(
                       h_num: rt.TH1,
                       h_den: rt.TH1,
                       option: str = "pois",
                       title: str = h_num.GetTitle()) -> rt.TRatioPlot:
        """
        Build and return a TRatioPlot for h_num / h_den (no drawing).
        """
        
        #creating a clone of h_num..... i'd be formatting some of its attributes
        h_num = h_num.Clone(h_num.GetName())
        h_num.SetTitle(title)
        rp = rt.TRatioPlot(h_num, h_den, option)
        
        return rp

def draw_ratio(
                rp: rt.TRatioPlot,
                title: str = "",
                canvas_name: str = "c",
                logy: bool = False,
                xlabel: str = "",
                ylabel: str = "",
                yratio_label: str = "",
                width: int = 800,
                height: int = 600) -> rt.TCanvas:
    """
    Draw a TRatioPlot onto a TCanvas, automatically closing any existing
    canvas with the same name to avoid ROOT auto-deletion crashes.

    Args:
        rp: TRatioPlot object (from make_two_ratio).
        canvas_name: identifier for the new canvas.
        title: canvas title.
        width, height: canvas dimensions.
        logy: if True, set upper pad to log scale.

    Returns:
        The newly created and drawn TCanvas.
    """
    # Close any existing canvas with this name
    for canv in list(rt.gROOT.GetListOfCanvases()):
        if canv.GetName() == canvas_name:
            canv.Close()

    # Create new canvas
    c = rt.TCanvas(canvas_name, title or canvas_name, width, height)
    if logy:
        c.SetLogy()

    # Draw ratio plot
    rp.Draw()
    c.Update()
    
    # #changing to upperpad and setting title
    # up = rp.GetUpperPad()
    # up.cd() 
    
    return c


In [34]:
%jsroot on
p = create_two_ratio(h_num, h_den)
c = draw_ratio(p, canvas_name="ratio", title="My Ratio", logy = True)
c.Draw()

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 43 points have been skipped
